In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import sys
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 1000)
import time

In additional to the train and test data, the Testra dataset come with several different types of information to do network incidents, these are event, resource, severity and log features. In this notebook, we try to reshape the original data into a format that can be used as input for machine learning pipeline  

In the dataset, location, event, resource, severity and log feature comes with value like "location 1", or "event_type 1", these categorical feature comes in string format. so the first we can we do is the input the data in away that we only keep the number following the word. This is done with a converter function during file reading.

In [ ]:
def str_to_num(string):
    return int(string.split(" ")[1])

train=pd.read_csv('../input/train.csv', converters={'location':str_to_num})
test=pd.read_csv('../input/test.csv', converters={'location':str_to_num})
event=pd.read_csv('../input/event_type.csv', converters={'event_type':str_to_num})
log_feature=pd.read_csv('../input/log_feature.csv', converters={'log_feature':str_to_num})
severity=pd.read_csv('../input/severity_type.csv', converters={'severity_type':str_to_num})
resource=pd.read_csv('../input/resource_type.csv', converters={'resource_type':str_to_num})

In [ ]:
event.head(3)

These data are inter-linked via the 'id' field. Each row in train and test data contain an unique id, the same with severity data. for resource, event and log feature, we have same id appearing multiple time in each file, basically the same id can have multiple resource types, event types and log features. For log feature we additional have a numerical volume value for a id and log feature pair. 

so the basicall strategy here is to shape the resource, event and log feature data in a way that each row corresponds on one unique id, in the same way as train, test and severity files. 

For event and resrouce data, this can be done simply by performing one-hot encoding using the get_dummies method from pandas.

In [ ]:
traintest=train.append(test)
traintest=traintest.merge(right=severity, on='id')


In [ ]:
resource_by_id=pd.get_dummies(resource,columns=['resource_type'])
resource_by_id=resource_by_id.groupby(['id']).sum().reset_index(drop=False)

event_by_id=pd.get_dummies(event,columns=['event_type'])
event_by_id=event_by_id.groupby(['id']).sum().reset_index(drop=False)

In [ ]:
resource_by_id.head(5)

For log feature data, this is slightly more complex, as we also need to capture the volume value associate to each id & log feature pair. So what we do here is for each id create columens for all the log feature with defaul value to be 0, and use the volume value if the corresponding pair of id and log feature is present in the log feature file.  

We do this by firstly construct a dictionary to hold all the id, log feature and volume information. And then we construct a numpy array with the shape to accomendate id and all the log feature columns - one unique id per row, and finally we fill values from the dictionary into the array accordingly to construct the resulted dataframe

In [ ]:
log_feature_dict={}

for row in log_feature.itertuples():
    if row.id not in log_feature_dict:
        log_feature_dict[row.id]={}
    if row.log_feature not in log_feature_dict[row.id]:
        log_feature_dict[row.id][row.log_feature]=row.volume

colnames=['id']
for i in range(1,387):
    colnames.append('log_feature_'+str(i))

log_feature_by_id_np=np.zeros((18552,387))
count=0
for key, feature_dict in log_feature_dict.items():
    log_feature_by_id_np[count, 0]=np.int(key)
    for feature, volume in feature_dict.items():
        log_feature_by_id_np[count, feature]=np.int(volume)
    count+=1
log_feature_by_id=pd.DataFrame(data=log_feature_by_id_np, columns=colnames, dtype=np.int)

In [ ]:
log_feature_by_id.head(3)

let's examine the resulted dataframes with the reshaped resource, event and log feature data 

In [ ]:
print(traintest.shape)
print(resource_by_id.shape)
print(event_by_id.shape)
print(log_feature_by_id.shape)

now we are ready to join the data up

In [ ]:
traintest=traintest.merge(right=severity, on='id')
print(traintest.shape)

traintest=traintest.merge(right=resource_by_id, on='id')
print(traintest.shape)

traintest=traintest.merge(right=event_by_id, on='id')
print(traintest.shape)

traintest=traintest.merge(right=log_feature_by_id, on='id')
print(traintest.shape)

final step! we split the data back into train and test input data for machine learning 

In [ ]:
train_input=traintest.loc[0:train.shape[0]-1]
print("train_input shape is", train_input.shape)

test_input=traintest.loc[train.shape[0]::]
print("test_inpue shape is", test_input.shape)